# Modelling
<u>Tests using the following models :</u>
* Linear regression
* Random forest regressor
* Ridge and Lasso Regularization (add on to linear modelling?)

<u> Tests using the following variables:</u>
* Weather variables (rain, temperature, windspeed)
* Time variables (Day of week, month, year, time of day, public holiday)
* Sensor environment variables:
    * Sensor_id
    * Betweenness of the street 
    * Buildings in proximity to the sensor
    * Landmarks in proximity to the sensor  
    * Furniture in proximity to the sensor    
    * Lights in proximity to the sensor   


Normalise variables: should this be with MinMax or StandardScaler??


Process:
* Keep only data from sensor's with relatively complete data
* Split data into training ( 75%) and test (25%)
* Define the models to use in testing (linear regression, random forest, xgboost)
* Define the error metrics to use in evaluating the model performance

In [17]:
# import copy
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
# from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, cross_validate, cross_val_predict
import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.metrics import classification_report, mean_squared_error,r2_score, accuracy_score, mean_absolute_error, mean_absolute_percentage_error
# from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import MinMaxScaler
# import time as thetime
# from xgboost import XGBClassifier, XGBRegressor
# from time import time
# from sklearn.inspection import permutation_importance
# from scipy import stats
# import math
# from eli5.sklearn import PermutationImportance

# from warnings import simplefilter
# simplefilter(action='ignore', category=FutureWarning)

# import multiprocessing

# # To display tables in HTML output
# from IPython.display import HTML, display

from Functions import *

## Read in formatted data

In [18]:
data = pd.read_csv("../Cleaned_data/formatted_data_for_modelling_allsensors.csv", index_col = False)

### Delete unneeded columns
We currently include data from all sensors (even incomplete ones)

In [19]:
data = data.drop(['sensor_id'],axis=1) # don't want this included
# Get rid of columns in which none of the sensors have a value
for column in data.columns:
    if np.nanmax(data[column]) ==0:
        del data[column]

In [20]:
# Filter columns using the regex pattern in function input
regex_pattern = 'buildings$|furniture$|landmarks$'
data = data[data.columns.drop(list(data.filter(regex=regex_pattern)))].copy()

### Add a random variable (to compare performance of other variables against)

In [21]:
rng = np.random.RandomState(seed=42)
data['random'] = np.random.random(size=len(data))
data["random_cat"] = rng.randint(3, size=data.shape[0])

## Prepare data for modelling 
### Split into predictor/predictand variables

In [22]:
# The predictor variables
Xfull = data.drop(['hourly_counts'], axis =1)
# The variable to be predicted
Yfull = data['hourly_counts'].values

### Store the (non Sin/Cos) time columns and then remove them
Need them later to segment the results by hour of the day

In [23]:
data_time_columns = Xfull[['day_of_month_num', 'time', 'weekday_num', 'time_of_day']]
Xfull = Xfull.drop(['day_of_month_num', 'time', 'weekday_num', 'time_of_day','year', 'month','day', 'datetime', 'month_num'],axis=1)

## Define model pipelines (linear regression, random forest and XGBoost)
Include process to scale the data

In [30]:
lr_model_pipeline = Pipeline(steps=[['scaler',StandardScaler()],['linear_regressor',LinearRegression()]])
rf_model_pipeline = Pipeline(steps=[['scaler',StandardScaler()],['rf_regressor', RandomForestRegressor(n_estimators = 10,random_state = 1, n_jobs = 32)]])
# xgb_model_pipeline = Pipeline(steps=[['scaler',StandardScaler()],['xgb_regressor',XGBRegressor(random_state=1, n_jobs = 200)]])
# et_model_pipeline = Pipeline(steps=[['scaler',StandardScaler()],['et_regressor',ExtraTreesRegressor (n_estimators = 500, random_state = 1, n_jobs = 32)]])

## Run models with cross-validation

### Define the error metrics for the cross-validation to return, and the parameters of the cross validatio

In [29]:
error_metrics = ['neg_mean_absolute_error', 'r2', 'neg_root_mean_squared_error']
cv_parameters = KFold(n_splits=10, random_state=1, shuffle=True)

### Define regex's to remove columns not needed in various splits of removing column

In [26]:
column_regex_dict = {'withsubtypes':'buildings$|furniture$|landmarks$'}
# #                      'nosubtyes':'buildings_|furniture_|landmarks_|sensor_id'}
# #                      'time_and_weather':'buildings|furniture|landmarks|h_|lights|avg_n_floors|betweenness',
# #                       'just_location_features':'buildings$|furniture$|landmarks$|school_holiday|public_holiday|Temp|Humidity|Pressure|Rain|WindSpeed|Sin|Cos'}

### Loop through each combination of the models, and the variables to include in the modelling

In [ ]:
# Dataframe to store the scores for each model
error_metric_scores = pd.DataFrame()

# Dictionary to store dataframes of feature importance scores
feature_importance_scores ={}

# models_dict = {"linear_regressor": lr_model_pipeline, "xgb_regressor":xgb_model_pipeline, 
#                "rf_regressor":rf_model_pipeline, "et_regressor":et_model_pipeline}
models_dict = {"rf_regressor":rf_model_pipeline}
for model_name,model in models_dict.items():
    for regex_name, regex in column_regex_dict.items():
        # Run the model: return the estimators and a dataframe containing evaluation metrics
        estimators, error_metrics_df, feature_list, predictions = run_model_with_cv_and_predict(
            model, model_name, error_metrics, cv_parameters, Xfull, Yfull, regex_name, regex) 
        # Add evaluation metric scores for this model to the dataframe containing the metrics for each model
        error_metric_scores = error_metric_scores.append(error_metrics_df)
        
        # Create dataframe of feature importances (no feature importances for linear regression)
        if model_name != 'linear_regressor':
            feature_importances = pd.DataFrame(index =[feature_list])
            for idx,estimator in enumerate(estimators):
                    feature_importances['Estimator{}'.format(idx)] = estimators[idx][model_name].feature_importances_
            feature_importance_scores["{}_{}".format(model_name, regex_name)] = feature_importances

Running rf_regressor model, variables include withsubtypes


### Plot the predicted vs actual values from the CV process
Within cross validation each data point is included in the test set only once and thus despite their beng multiple cross-validation folds, each true value of Y has only one associated prediction 

In [ ]:
# fig, ax = plt.subplots()
# using_datashader(ax, Yfull, predictions, 'log')
# ax.plot([Yfull.min(), Yfull.max()], [Yfull.min(), Yfull.max()], c='k', lw=0.5)
# ax.set_ylabel("Predicted Values", size=10)
# ax.set_xlabel("Actual Values", size=10)
# ax.xaxis.set_tick_params(labelsize='xx-large')
# ax.yaxis.set_tick_params(labelsize='xx-large')
# ax.set_xlim([0, 2000])
# ax.set_ylim([0, 2000]);

### Print the accuracy scores

In [ ]:
# error_metric_scores

### Feature importances from within cross-validation
If reporting feature importances from the model, then would use those from fitting the final model on the full dataset. However, this is useful as a measure of the stability of the feature importances that the model reports

In [ ]:
feature_importances_df = feature_importance_scores["rf_regressor_withsubtypes"].copy()
feature_importances_df.reset_index(inplace=True)
feature_importances_df.rename(columns={'level_0':'Variable'},inplace=True)

#### Get the most important top 10 columns for each estimator

In [ ]:
# important_columns=pd.DataFrame()
# for column in feature_importances_df.columns[1:]:
#     this_col = feature_importances_df[['Variable', column]]
#     important_columns[column] = this_col.sort_values(column, ascending = False)['Variable'].tolist()[0:10]
# important_columns    